In [1]:
from tvDatafeed import TvDatafeed, Interval
import mysql.connector
import pandas as pd

In [3]:
def query(query, host="ideatrade.serveftp.net", port=51410, user="Wasan@sun", password=")Ai_C40bj]6ssI9W", database="db_ideatrade"):
    with mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            columns = [column[0] for column in cursor.description]
            result = cursor.fetchall()

    return pd.DataFrame(columns=columns, data=result)

In [4]:
to_database_vm = query("SELECT symbol, exchange FROM allsymbol_global;")
to_database_vm.to_csv("to_database_vm.csv", index=False)

In [5]:
# ! pip install gspread google-auth google-auth-oauthlib google-auth-httplib2
# ! pip install --upgrade certifi
# ! pip install mysql-connector-python
# ! pip install google-auth

import pandas as pd
pd.options.display.float_format = "{:,.3f}".format

import numpy as np
import datetime
import mysql.connector
import yfinance as yf
import requests

import warnings
warnings.filterwarnings('ignore')
from functools import lru_cache

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'tahoma'
import matplotlib.ticker as ticker

import gspread
from google.oauth2.service_account import Credentials
#import gspread_dataframe as gd

In [6]:
from datetime import datetime, timedelta
def get_data():
    # โหลดข้อมูลรับรอง JSON จากไฟล์ที่คุณดาวน์โหลด
    credentials = Credentials.from_service_account_file('keys.json', scopes=['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])

    # ตั้งค่าการเชื่อมต่อ gspread ด้วยข้อมูลรับรอง
    gc = gspread.service_account(filename='keys.json')

    # เปิดสเปรดชีตของ Google Sheets โดยใช้ชื่อหรือ URL
    spreadsheet = gc.open('stores_shopee')
 
    # เลือกชีตที่ต้องการใน Google Sheets
    worksheet = spreadsheet.worksheet('ชีต2')

    # อ่านข้อมูลจากชีตเป็นรายการของดิกชันนารี
    data = worksheet.get_all_records()

    # สร้าง DataFrame จากข้อมูล
    df = pd.DataFrame(data)

    return df
df = get_data()

In [7]:

vm = 1
df_vm = df[df['vm'] == vm]
df_vm.reset_index(drop=True, inplace=True)


In [73]:
df_vm

,symbol,exchange,vm
0,ZZZ,NEO,1
1,ZZG,BER,1
2,ZYUS,ASX,1
3,ZYME,NASDAQ,1
4,ZYDUSLIFE,BSE,1
...,...,...,...
26460,23,SZSE,1
26461,19,SZSE,1
26462,12,SZSE,1
26463,8,SZSE,1


In [63]:
from tvDatafeed import TvDatafeed, Interval
import mysql.connector
import pandas as pd
from tqdm import tqdm

tv = TvDatafeed()

def get_1min(tv, symbol, exchange):
    try:
        # Convert numeric symbol to string
        if isinstance(symbol, (int, float)):
            symbol = str(symbol)

        # Check if symbol and exchange are strings before using strip()
        if isinstance(symbol, str) and isinstance(exchange, str):
            symbol = symbol.strip()
            exchange = exchange.strip()

            res = tv.get_hist(symbol=symbol, exchange=exchange, interval=Interval.in_1_minute, n_bars=10000)
            return res
        else:
            print(f"Skipping {symbol} on {exchange}: Symbol and exchange must be strings.")
            return pd.DataFrame()  # Return an empty DataFrame for symbols that couldn't be retrieved
    except Exception as e:
        print(f"Error retrieving data for {symbol} on {exchange}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame for the symbol that couldn't be retrieved


# Execute the query to get symbols and exchanges
sym_ex = df_vm[['symbol', 'exchange']]

# Iterate through symbols and exchanges, and fetch data for each combination
all_data = []
s= 0
for ind, row in tqdm(sym_ex.iterrows(), "Getting TV"):
    symbol = row['symbol']
    exchange = row['exchange']
    data = get_1min(tv, symbol, exchange)
    s += 1
    if data is not None and not data.empty:
        all_data.append(data)
    

# Concatenate the data for all symbols and exchanges
result = pd.concat(all_data)

# Now, 'result' contains the concatenated historical data for all symbols in every exchange
# You can proceed to use or store this data as needed


Getting TV: 49it [01:17,  1.53s/it]ERROR:tvDatafeed.main:The read operation timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol
Getting TV: 49it [01:22,  1.68s/it]


In [57]:
df_symbol = df_vm[['symbol']].drop_duplicates()
df_exchage = df_vm[['exchange']].drop_duplicates()


In [62]:

count_int = 0
count_str = 0
count_t = 0
lst = []

df_exchage = df_vm[['exchange']].drop_duplicates()

for i in df_symbol['symbol']:
    if type(i) == int:
        count_int += 1
    elif type(i) == str:
        
        count_str += 1
    else :
        count_t += 1
        print(type(i))
        lst.append(i)
# count_int, count_str, count_t
# lst


<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


In [55]:
count_int, count_str, count_t

(0, 91, 0)

In [49]:
lst[0]

inf

In [69]:

result_symbol = result[['symbol']].drop_duplicates()
result_symbol

,symbol
datetime,
2022-07-11 20:44:00,NEO:ZZZ
2009-02-02 14:40:00,BER:ZZG
2021-12-20 06:04:00,ASX:ZYUS
2023-11-13 21:30:00,NASDAQ:ZYME
2023-11-28 10:45:00,BSE:ZYDUSLIFE
2011-05-02 14:00:00,FWB:ZYA
2022-10-31 23:12:00,BER:ZYA
2015-06-29 20:30:00,TSX:ZXM
2021-01-27 22:49:00,NEO:ZWT


symbol            ZSL
exchange    NYSE ARCA
Name: 49, dtype: object

In [66]:
result_symbol.count()

symbol    49
dtype: int64

In [77]:
# index
nifty_index_data = tv.get_hist(symbol='NIFTY',exchange='NSE',interval=Interval.in_1_minute,n_bars=10000)

# futures continuous contract
nifty_futures_data = tv.get_hist(symbol='NIFTY',exchange='NSE',interval=Interval.in_1_hour,n_bars=1000,fut_contract=1)

# crudeoil
crudeoil_data = tv.get_hist(symbol='CRUDEOIL',exchange='MCX',interval=Interval.in_1_hour,n_bars=5000,fut_contract=1)

# downloading data for extended market hours
extended_price_data = tv.get_hist(symbol="EICHERMOT",exchange="NSE",interval=Interval.in_1_hour,n_bars=500, extended_session=False)


ERROR:tvDatafeed.main:The read operation timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


In [75]:
nifty_futures_data

,symbol,open,high,low,close,volume
datetime,,,,,,
2023-05-29 12:45:00,NSE:NIFTY1!,"18,711.000","18,717.500","18,691.500","18,697.850","310,150.000"
2023-05-29 13:45:00,NSE:NIFTY1!,"18,697.850","18,701.500","18,657.050","18,671.750","687,450.000"
2023-05-29 14:45:00,NSE:NIFTY1!,"18,671.750","18,706.000","18,670.900","18,703.000","543,200.000"
2023-05-29 15:45:00,NSE:NIFTY1!,"18,703.000","18,705.000","18,651.050","18,670.050","793,300.000"
2023-05-29 16:45:00,NSE:NIFTY1!,"18,670.000","18,676.000","18,665.000","18,675.000","321,800.000"
...,...,...,...,...,...,...
2023-12-22 12:45:00,NSE:NIFTY1!,"21,449.500","21,459.300","21,424.900","21,427.400","776,050.000"
2023-12-22 13:45:00,NSE:NIFTY1!,"21,428.550","21,435.000","21,344.750","21,393.550","1,171,750.000"
2023-12-22 14:45:00,NSE:NIFTY1!,"21,394.000","21,415.000","21,285.000","21,368.000","1,434,800.000"


In [78]:
nifty_index_data

,symbol,open,high,low,close,volume
datetime,,,,,,
2023-12-04 10:45:00,NSE:NIFTY,"20,602.500","20,602.500","20,537.200","20,543.000","13,331,122.000"
2023-12-04 10:46:00,NSE:NIFTY,"20,544.950","20,557.600","20,539.700","20,555.100","5,756,084.000"
2023-12-04 10:47:00,NSE:NIFTY,"20,553.600","20,559.000","20,547.450","20,547.450","3,445,582.000"
2023-12-04 10:48:00,NSE:NIFTY,"20,549.600","20,550.950","20,531.100","20,534.200","2,847,064.000"
2023-12-04 10:49:00,NSE:NIFTY,"20,532.200","20,541.550","20,528.900","20,530.550","4,538,630.000"
...,...,...,...,...,...,...
2023-12-22 16:55:00,NSE:NIFTY,"21,343.400","21,343.600","21,330.300","21,330.650","1,133,851.000"
2023-12-22 16:56:00,NSE:NIFTY,"21,329.500","21,332.650","21,329.300","21,331.400","909,391.000"
2023-12-22 16:57:00,NSE:NIFTY,"21,331.950","21,335.700","21,328.450","21,334.950","1,216,267.000"
